In [1]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def check_fingerprint(wb, matr):
    ws = wb.worksheets[0]
    if ws["BB10"].value == int(matr):
        return "OK"
    else:
        return "NOK"

def check_arrayformula(wb, matr):
    try:
        response = "        "
        ws = wb.worksheets[0]
        if ws.array_formulae != {}:
            response = response[:1] + "1" + response[2:]
        
        ws = wb.worksheets[1]
        if ws.array_formulae != {}:
            response = response[:3] + "2" + response[4:]

        ws = wb.worksheets[2]
        if ws.array_formulae != {}:
            response = response[:5] + "3" + response[6:]

        return response
    except:
        print("FAIL:", matr, "check Matrix_Formula")
        return "NOK"
        
def check_exercise_1_1(wb):
    ws = wb.worksheets[0]
    points = 0
    # Check if background color is FH-Mint
    if ".00" in ws["B4"].number_format and "€" in ws["B4"].number_format and "[Red]" in ws["E4"].number_format:
        points += 1
    if " J" in ws["A5"].number_format:
        points += 1
    if ws["M3"].fill.start_color.index == "FF00B1AC" and ws["K1"].font.size == 16:
        points += 1
    return points

def check_exercise_1_2(wb, wb_val):
    # Check formulas
    ws = wb.worksheets[0]
    points = 0
    # Check formulas
    if ws["B4"].value != None:
        formula = ws["B4"].value
        formula = formula.split(",")
        if "=PMT" in formula[0]:
            points += 1
        if "B$3/12" in formula[0]:
            points += 1
        if "$A4*12" in formula[1] or "12*$A4" in formula[1]:
            points += 1

    if ws["L4"].value != None:
        formula = ws["L4"].value
        if "12" in formula:
            points += 1
        if "$A4" in formula or "$K4" in formula:
            points += 1
        if "B4" in formula:
            points += 1
  
    return points

def check_exercise_2_1(wb):
    ws = wb.worksheets[1]
    points = 0
    # Check if background color is FH-Mint
    if ws["A11"].fill.start_color.index == ws["D6"].fill.start_color.index == "FF00B1AC":
        points += 1
    # Check if number-format in B2 is %
    if "%" in ws["B2"].number_format:
        points += 1 

    return points

def check_exercise_2_2(wb, wb_val):
    # Check formulas
    ws = wb.worksheets[1]
    points = 0
    # Check formulas
    if ws["B7"].value != None:
        formula = ws["B7"].value
        formula = formula.split(",")
        if "=IPMT" in formula[0]:
            points += 1
            if "B$2" in formula[0]:
                points += 1
            if "A7" in formula[1]:
                points += 1
            if "B$3" in formula[2]:
                points += 1
    if ws["C7"].value != None:
        if "=PPMT" in ws["C7"].value:
            points += 1

    ws_val = wb_val.worksheets[1]
    for row in range(7, ws_val.max_row + 2):
        if ws_val["A" + str(row)].value != None:
            max_row = row
    if ws_val["E" + str(max_row)].value != None:
        if round(ws_val["E" + str(max_row)].value, 2) == 0:
            points += 1
    if round(ws_val["D7"].value, 2) == round(ws_val["D8"].value, 2) == round(ws_val["D9"].value, 2):
        points += 1

    return points

def check_exercise_3_1(wb):
    ws = wb.worksheets[2]
    points = 0
    # check number-format of G2
    if ".00" in ws["G2"].number_format and "€" in ws["G2"].number_format and "[Red]" in ws["G2"].number_format:
        points += 1
    if " J" in ws["F10"].number_format:
        points += 1
    if ws["D7"].fill.start_color.index == ws["F2"].fill.start_color.index == ws["K13"].fill.start_color.index == "FF00B1AC":
        points += 1
    return points

def check_exercise_3_2(wb, wb_val):
    # Check formulas
    ws = wb.worksheets[2]
    points = 0

    # Check, if the formula SLN() is used in C9
    if ws["C9"].value != None:
        formula = ws["C9"].value
        if "=SLN(" in formula:
            points += 1

    # Check, if the formula SYD() is used in H9
    if ws["H9"].value != None:
        formula = ws["H9"].value
        formula = formula.split(",")
        if "=SYD(" in formula[0]:
            points += 10
            if "F9" in formula[3]:
                points += 100

    # Check, if formula N9 contains a reasonable formula
    if ws["N9"].value != None:
        formula = ws["N9"].value
        if "L$2" in formula and "L$5" in formula and "L9" in formula:
            points += 1000

    ws_val = ws_val = wb_val.worksheets[2]
    for row in range(7, ws_val.max_row + 2):
        if ws_val["A" + str(row)].value != None:
            max_row = row
    if ws_val["D" + str(max_row)].value != None and ws_val["I" + str(max_row)].value != None and ws_val["O" + str(max_row)].value != None:
        if round(ws_val["D" + str(max_row)].value, 2) == round(ws_val["I" + str(max_row)].value, 2) == round(ws_val["O" + str(max_row)].value, 2) == 0:
            points += 10000

    if round(ws_val["N8"].value, 2) == round(ws_val["L2"].value * ws_val["L8"].value / ws_val["L5"].value, 2):
        points += 100000
        







    return points
    
def check_workbook(matr, wb, wb_val):
    points = [matr]
    points.append(check_fingerprint(wb, matr))
    points.append(check_arrayformula(wb, matr))
    points.append(check_arrayformula(wb, matr))
    points.append(check_exercise_1_1(wb))
    points.append(check_exercise_1_2(wb, wb_val))
    points.append(check_exercise_2_1(wb))
    points.append(check_exercise_2_2(wb, wb_val))
    points.append(check_exercise_3_1(wb))
    points.append(check_exercise_3_2(wb, wb_val))
    

  
    return points



l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)
for point in l_points:
    print(point)
header = ["MatrNr", "Check", "Array-Formula", "1 Format", "1 Formel", "2 Format", "2 Formel", "3 Format", "3 Formel"] 
common.result_into_file(l_points, header)


[3580000, 'NOK', '        ', '        ', 2, 6, 2, 7, 1, 110111]
